In [1]:
import os 
import sys 

sys.path.append('C:/Program Files/Anaconda3/Lib/site-packages')
sys.path.append('C:/Program Files (x86)/Microsoft Visual Studio/Shared/Anaconda3_64/lib/site-packages')


import time 
import psutil 
import logging 
import pyautogui as pgui 
from datetime import datetime 
import pyautocad

In [21]:
#------------------------------------------------------------------------------- 
# Functions  
 
def extractSelectedFormat(directory_path, file_extension='dwg'):       
    # This function extracts all the files of the selected format from the 
    # directory specified in the variable data_wd. Format is recognized through the extension. 
    # not important
     
    files = os.listdir(directory_path)
    files = [file for file in files if file.split('.')[1] == file_extension] 
    files = [os.path.join(directory_path, file) for file in files] 
    return files 
         
     
def checkExist(directory_path ,file_name):      
    # Checks if the file file_name exists 
    # not important
     
    files = os.listdir(directory_path) 
    printed = file_name in files 
    return printed 

def checkProgram(p_name):  # why need it?
    # Checks if the program p_name is currently running. 
    # 
    # Arguments: 
    #   p_name - string - name of the program. Example: 'acad.exe' 
    # Returns: True if the program is running, False otherwise. 
    # 
     
    processes = [psutil.Process(p) for p in psutil.pids()] 
    processes = [p.name().lower() for p in processes] 
    if p_name in processes: 
        return True 
    else: 
        return False 


def print_file_pdf(file_path, directory_path, logger): 
    # Prints to pdf all the dwg files in the path specified 
    #   logger - a logger. 
    #   Returns: exit_status - int - 1 if Critical error occurred, 0 otherwise 
    #  
     
    file_name = file_path.split("\\")[-1]                        # Get name of the file 
    file_name_pdf = file_name.replace("dwg", "pdf")              # Get name of the output file 
    is_printed = checkExist(directory_path, file_name_pdf)       # Check if already existing 
    exit_status = 0                                              # 1 only if critical error occurs 
     
    if not is_printed:     
         
        program_runs = checkProgram("acad.exe")   
        # 서버에서 autocad를 실행시키거나, 프로그램이 클라이언트로 와야함 
        # but client에서 실행하면 DRM이 문제가 생김
        # Opens file with default program (Autocad for .dwg files). 
        # If opening fails, returns 1, else 0. 
        exit_status = os.system(" ".join(["start", file_path])) 
         
        if exit_status == 0: 
            if program_runs: 
                time.sleep(5) 
            else: 
                # Autocad is slow to start up if it is not already running. 
                time.sleep(30) 
             
            print(" ".join(["Now printing file", file_name])) 
             
            # Starting printing sequence 
            pgui.hotkey("ctrl","p") 
            pgui.typewrite("mi") 
            pgui.keyDown("enter") 
            pgui.keyUp("enter") 
            time.sleep(1) 
            pgui.typewrite(file_name_pdf) 
            pgui.keyDown("enter") 
            pgui.keyUp("enter") 
            time.sleep(3) 
            # Closing current drawing 
            pgui.hotkey("ctrl", "F4") 
            pgui.keyDown("n") 
            pgui.keyUp("n") 
         
            is_printed = check_file_exist(directory_path, file_name_pdf) 
            if is_printed: 
                logger.log(logging.INFO, " ".join(["Printed", file_name])) 
            else: 
                logger.log(logging.CRITICAL, " ".join(["NOT Printed", file_name])) 
                exit_status = 1 
        else: 
            logger.log(logging.ERROR, "".join(["Could not open ", file_name])) 
            logger.log(logging.ERROR, " ".join(["NOT Printed", file_name])) 
    else: 
        logger.log(logging.INFO, " ".join(["Already printed", file_name])) 
      
    time.sleep(1) 
     
    return exit_status 

 
def handle_critical_errors(error_count, check_at_count=2): 
     
    # Two or more critical errors may indicate a severe malfunctioning in the  
    # program. This function displays a confirmation box asking the user if 
    # they want to continue or stop the program after check_at_count critical 
    # errors occurred. 
    # 
    # Arguments: 
    #   error_count - string - count of the critical errors. 
    #   check_at_count - int - check after check_at_count critical errors occurred. 
    #  
    #   Returns: void 
    #  
    if error_count >= check_at_count: 
        value = pgui.confirm("""2 or more critical errors have occurred. 
                             Would you like to continue (click "ok") 
                             or cancel?""") 
        if value == 'Cancel': 
            sys.exit() 




In [20]:
#------------------------------------------------------------------------------- 
# Main 

 
if __name__ == '__main__': 
     
    # Setting input data directory 
    data_wd = "C:\\Users\\mando\\Desktop\\ocr\\도면" 
    full_file_path = extractSelectedFormat(data_wd) 
    print("\nFollowing files will be printed: {}".format([full_file_path])) 
     
    pgui.PAUSE = 0.5 
    time_stamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S') 
    error_count = 0 
    logging.basicConfig(filename = 'logfile.log', level=logging.DEBUG) 
    logger = logging.getLogger('DWG Printer bot V. {}'.format(VERSION))  # version from autocad
    logger.log(logging.INFO, 
               "Program ran at {}. Directory chosen: {}".format(time_stamp, data_wd)) 

 
    # Start printing 
     
    for file in full_file_path: 
        try: 
            error_count += print_file_pdf(file, data_wd, logger) 
            print("Exiting current printing call...\n") 
            handle_critical_errors(error_count, 2) 
        except Exception as e: 
            print(str(e)) 
    
    final_time_stamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S') 
    logger.log(logging.INFO, "Program ended at {}\n\n".format(final_time_stamp)) 
    pgui.alert("Job ended!") 



Following files will be printed: [['C:\\Users\\mando\\Desktop\\ocr\\도면\\BE600-EAC-XXI_SK3.dwg', 'C:\\Users\\mando\\Desktop\\ocr\\도면\\BE600EBYXX_F.dwg', 'C:\\Users\\mando\\Desktop\\ocr\\도면\\BE600EBYXX_F_00.dwg', 'C:\\Users\\mando\\Desktop\\ocr\\도면\\BE600PY4XX_G.dwg', 'C:\\Users\\mando\\Desktop\\ocr\\도면\\BH290019XX_Y.dwg', 'C:\\Users\\mando\\Desktop\\ocr\\도면\\BH290019XX_Y_00.dwg', 'C:\\Users\\mando\\Desktop\\ocr\\도면\\BH290023XX_P.dwg', 'C:\\Users\\mando\\Desktop\\ocr\\도면\\BH290023XX_P_00.dwg', 'C:\\Users\\mando\\Desktop\\ocr\\도면\\도면 개선안_00.dwg']]


NameError: name 'VERSION' is not defined

In [ ]:
# where is VERSION? , cant read by DRM?

## choice2

In [8]:
import dxfgrabber 
import cv2 
import numpy as np 
import sys 


In [9]:
def getMinXY(shapes): 
    minX, minY = 99999, 99999 
    for shape in shapes: 
        if shape.dxftype == 'LINE': 
            minX = min(minX, shape.start[0], shape.end[0]) 
            minY = min(minY, shape.start[1], shape.end[1]) 
        elif shape.dxftype == 'ARC': 
            minX = min(minX, shape.center[0]) 
            minY = min(minY, shape.center[1]) 

 
    return minX, minY 


In [10]:
def getMaxXY(shapes): 
    maxX, maxY = -99999, -99999 
    for shape in shapes: 
        if shape.dxftype == 'LINE': 
            maxX = max(maxX, shape.start[0], shape.end[0]) 
            maxY = max(maxY, shape.start[1], shape.end[1]) 
        elif shape.dxftype == 'ARC': 
            maxX = max(maxX, shape.center[0]) 
            maxY = max(maxY, shape.center[1]) 

 
    return maxX, maxY 


In [12]:
def getXY(point): 
    x, y = point[0], point[1] 
    return int(x-minX+100), int(abs(y-maxY)+100) 

 
dxf = dxfgrabber.readfile(sys.argv[1]) 
shapes = dxf.entities.get_entities() 
minX, minY = getMinXY(shapes) 
maxX, maxY = getMaxXY(shapes) 
#baseX, baseY = absdiff(minX, 0), absdiff(minY, 0) 
#absMaxX, absMaxY = absdiff(maxX, 0), absdiff(maxY, 0) 

 

print(maxX, maxY) 
print(minX, minY) 
#print(absMaxX, absMaxY) 
#print(baseX, baseY) 


FileNotFoundError: [Errno 2] No such file or directory: '-f'

In [14]:
canvas = np.zeros((512,512,3), np.uint8) 
for shape in shapes: 
    if shape.dxftype == 'LINE': 
        x1, y1 = getXY(shape.start) 
        x2, y2 = getXY(shape.end) 
        canvas = cv2.line(canvas, (x1,y1), (x2,y2), (255, 0, 255), 1) 
    elif shape.dxftype == 'ARC': 
        centerX, centerY = getXY(shape.center)         
        if (shape.start_angle > 180) and (shape.end_angle < shape.start_angle): 
            canvas = cv2.ellipse(canvas, (centerX, centerY), (int(shape.radius), int(shape.radius)), 180, int(shape.start_angle) - 180, 180 + int(shape.end_angle), (0, 0, 255), 1) 
        else: 
            canvas = cv2.ellipse(canvas, (centerX, centerY), (int(shape.radius), int(shape.radius)), 0, int(360 - shape.start_angle), int(360 - shape.end_angle), (0, 0, 255), 1) 
    cv2.imshow('test', canvas) 
    cv2.waitKey(50) 

# To save canvas use cv2.imwrite() 
#cv2.imwrite('res.png', canvas) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 


NameError: name 'shapes' is not defined